In [1]:
import re
from typing import List, Optional
from langchain_core.documents import Document

def load_txt(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def extract_chapter_meta(chapter_num: str, chapter_title: str) -> dict:
    """
    chapter_num이 '01' → level 1, parent 없음
    chapter_num이 '01-02' → level 2, parent '01'
    """
    if "-" in chapter_num:
        parent = chapter_num.split("-")[0]
        level = 2
    else:
        parent = None
        level = 1
    return {
        "chapter": chapter_num,
        "title": chapter_title,
        "level": level,
        "parent": parent
    }

def split_by_chapter(text: str) -> List[Document]:
    """
    Wikidocs 형식 텍스트를 챕터별로 나누고 계층 정보를 포함한 Document로 변환
    """
    pattern = r"(?=^---\s+(\d{2}(?:-\d{2})?)\.\s+(.*?)\s+---)"
    matches = list(re.finditer(pattern, text, flags=re.MULTILINE))

    documents = []
    for i, match in enumerate(matches):
        start = match.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

        chapter_num = match.group(1)
        chapter_title = match.group(2).strip()
        chapter_text = text[start:end].strip()

        metadata = extract_chapter_meta(chapter_num, chapter_title)

        doc = Document(page_content=chapter_text, metadata=metadata)
        documents.append(doc)

    return documents

def load_and_split_wikidocs(path: str) -> List[Document]:
    text = load_txt(path)
    return split_by_chapter(text)

def filter_chapters(
    documents: List[Document],
    level: Optional[int] = None,
    parent: Optional[str] = None,
    contains_title: Optional[str] = None
) -> List[Document]:
    """
    챕터 리스트에서 조건에 맞는 챕터만 필터링
    - level: 1 (대챕터), 2 (소챕터)
    - parent: '01' 등 상위 챕터 번호
    - contains_title: 제목 키워드 포함 여부
    """
    filtered = []
    for doc in documents:
        meta = doc.metadata
        if level and meta["level"] != level:
            continue
        if parent and meta["parent"] != parent:
            continue
        if contains_title and contains_title not in meta["title"]:
            continue
        filtered.append(doc)
    return filtered

In [2]:
first_docs = load_and_split_wikidocs("C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_01.txt")

# 소챕터만 추출 (level 2)
subchapters = filter_chapters(first_docs, level=2)

# 대챕터 02의 모든 소챕터 추출
chap02_children = filter_chapters(first_docs, parent="02")

# "설치"라는 단어를 포함하는 챕터만 추출
install_sections = filter_chapters(first_docs, contains_title="설치")

print(f"📌 총 문서 수: {len(first_docs)}")
print(f"🔹 소챕터 수: {len(subchapters)}")
print(f"🔹 02번 챕터 하위 수: {len(chap02_children)}")
print(f"🔹 '설치' 포함 제목 수: {len(install_sections)}")

📌 총 문서 수: 165
🔹 소챕터 수: 0
🔹 02번 챕터 하위 수: 0
🔹 '설치' 포함 제목 수: 1


In [3]:
second_docs = load_and_split_wikidocs("C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_02.txt")

# 소챕터만 추출 (level 2)
subchapters = filter_chapters(second_docs, level=2)

# 대챕터 02의 모든 소챕터 추출
chap02_children = filter_chapters(second_docs, parent="02")

# "설치"라는 단어를 포함하는 챕터만 추출
install_sections = filter_chapters(second_docs, contains_title="설치")

print(f"📌 총 문서 수: {len(second_docs)}")
print(f"🔹 소챕터 수: {len(subchapters)}")
print(f"🔹 02번 챕터 하위 수: {len(chap02_children)}")
print(f"🔹 '설치' 포함 제목 수: {len(install_sections)}")

📌 총 문서 수: 27
🔹 소챕터 수: 0
🔹 02번 챕터 하위 수: 0
🔹 '설치' 포함 제목 수: 0


In [4]:
third_docs = load_and_split_wikidocs("C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_03.txt")

# 소챕터만 추출 (level 2)
subchapters = filter_chapters(third_docs, level=2)

# 대챕터 02의 모든 소챕터 추출
chap02_children = filter_chapters(third_docs, parent="02")

# "설치"라는 단어를 포함하는 챕터만 추출
install_sections = filter_chapters(third_docs, contains_title="설치")

print(f"📌 총 문서 수: {len(third_docs)}")
print(f"🔹 소챕터 수: {len(subchapters)}")
print(f"🔹 02번 챕터 하위 수: {len(chap02_children)}")
print(f"🔹 '설치' 포함 제목 수: {len(install_sections)}")

📌 총 문서 수: 33
🔹 소챕터 수: 4
🔹 02번 챕터 하위 수: 0
🔹 '설치' 포함 제목 수: 0


In [5]:
print(len(first_docs))
print(len(second_docs))
print(len(third_docs))

165
27
33


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 초기화 (하나의 스플리터만 사용)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 청크 크기
    chunk_overlap=50,  # 청크 간 중복
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # 분할 기준
)

first_splits = []
second_splits = []
third_splits = []

for doc in first_docs:
    splits = splitter.split_text(doc.page_content)
    first_splits.extend(splits)
    # print(first_splits)

for doc in second_docs:
    splits = splitter.split_text(doc.page_content)
    second_splits.extend(splits)
    # print(second_splits)

for doc in third_docs:
    splits = splitter.split_text(doc.page_content)
    third_splits.extend(splits)
    # print(third_splits)

print(f"첫 번째 문서 청크 수: {len(first_splits)}")
print(f"두 번째 문서 청크 수: {len(second_splits)}")
print(f"세 번째 문서 청크 수: {len(third_splits)}")

첫 번째 문서 청크 수: 4746
두 번째 문서 청크 수: 1983
세 번째 문서 청크 수: 2964


In [7]:
from sentence_transformers import SentenceTransformer
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
import shutil

# OpenAI 임베딩 모델 초기화
from dotenv import load_dotenv

# 환경 변수에서 API 키 로드
load_dotenv()

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings()  # OpenAI 임베딩 모델 지정

# DB 디렉토리 존재 여부 확인 및 삭제
if os.path.exists(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\first_db"):
    shutil.rmtree(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\first_db")
if os.path.exists(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\second_db"):
    shutil.rmtree(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\second_db")
if os.path.exists(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\third_db"):
    shutil.rmtree(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\third_db")

# 첫 번째 문서 DB 생성
first_db = Chroma.from_texts(
    texts=first_splits,
    embedding=embeddings,
    persist_directory=r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\first_db"
)
first_db.persist()

# # 두 번째 문서 DB 생성
# second_db = Chroma.from_texts(
#     texts=second_splits,
#     embedding=embeddings,
#     persist_directory=r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\second_db"
# )
# second_db.persist()

# # 세 번째 문서 DB 생성
# third_db = Chroma.from_texts(
#     texts=third_splits,
#     embedding=embeddings,
#     persist_directory=r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\third_db"
# )
# third_db.persist()

C:\Users\user\AppData\Local\Temp\ipykernel_4640\1310670779.py:30: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  first_db.persist()


In [8]:
first_db.as_retriever(search_type="similarity", search_kwargs={"k": 10}).invoke('import pandas')


[Document(metadata={}, page_content='# 원하는 Pandas DataFrame을 정의합니다.\ndf = pd.read_csv("./data/titanic.csv")\ndf.head()'),
 Document(metadata={}, page_content='PassengerId\nSurvived\nPclass\nName\nSex\nAge\nSibSp\nParch\nTicket\nFare\nCabin\nEmbarked\n1\n0\n3\nBraund, Mr. Owen Harris\nmale\n22\n1\n0\nA/5 21171\n7.25\nS\n2\n1\n1\nCumings, Mrs. John Bradley (Florence Briggs Thayer)\nfemale\n38\n1\n0\nPC 17599\n71.2833\nC85\nC\n3\n1\n3\nHeikkinen, Miss. Laina\nfemale\nDataFrameLoader\nPandas는 Python 프로그래밍 언어를 위한 오픈 소스 데이터 분석 및 조작 도구입니다. 이 라이브러리는 데이터 과학, 머신러닝, 그리고 다양한 분야의 데이터 작업에 널리 사용되고 있습니다.\nimport pandas as pd\n# CSV 파일 읽기\ndf = pd.read_csv("./data/titanic.csv")\n첫 5개 행을 조회합니다.'),
 Document(metadata={}, page_content="연관키워드: 딥러닝, 자연어 처리, 시퀀스 모델링\n판다스 (Pandas)\nMetadata: {'source': './data/appendix-keywords.txt', 'id': 10, 'relevance_score': 0.9997084}"),
 Document(metadata={}, page_content='# !pip install -qU langchain-teddynote\nfrom langchain_teddynote import logging\n# 프로젝트 이름을 입력합니다.

In [9]:
# import re
# from typing import List
# from langchain_core.documents import Document
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# def load_txt(path: str) -> str:
#     """
#     텍스트 파일을 UTF-8로 로드
#     """
#     with open(path, "r", encoding="utf-8") as f:
#         return f.read()

# def extract_chapter_meta(chapter_num: str, chapter_title: str) -> dict:
#     """
#     개선된 버전: 제목 내 [카테고리] → parent
#     """
#     # [카테고리]가 존재하면 parent로 추정
#     category_match = re.match(r"\[(.*?)\]", chapter_title)
#     if category_match:
#         parent = category_match.group(1)
#     else:
#         parent = None

#     return {
#         "chapter": chapter_num,
#         "title": chapter_title,
#         "level": 1,  # 실제 파일에는 대챕터만 존재
#         "parent": parent
#     }

# def protect_code_blocks(text: str) -> str:
#     """
#     ```로 감싸진 코드 블록을 <CODE_BLOCK>으로 감싸 보존
#     """
#     code_pattern = re.compile(r"```.*?\n.*?```", re.DOTALL)
#     protected = []
#     last_end = 0

#     for match in code_pattern.finditer(text):
#         start, end = match.span()
#         protected.append(text[last_end:start])
#         code = match.group()
#         protected.append(f"\n<CODE_BLOCK>\n{code}\n</CODE_BLOCK>\n")
#         last_end = end

#     protected.append(text[last_end:])
#     return "".join(protected)

# def split_by_chapter_with_code(text: str) -> List[Document]:
#     """
#     챕터 헤더(--- 01. 제목 ---) 기준으로 분할하되, 각 챕터 내 코드블록 보존
#     """
#     pattern = r"(?=^---\s+(\d{2}(?:-\d{2})?)\.\s+(.*?)\s+---)"
#     matches = list(re.finditer(pattern, text, flags=re.MULTILINE))

#     documents = []
#     for i, match in enumerate(matches):
#         start = match.start()
#         end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

#         chapter_num = match.group(1)
#         chapter_title = match.group(2).strip()
#         chapter_text = text[start:end].strip()

#         protected_text = protect_code_blocks(chapter_text)
#         metadata = extract_chapter_meta(chapter_num, chapter_title)

#         documents.append(Document(page_content=protected_text, metadata=metadata))

#     return documents

# def process_wikidocs_files(paths: List[str]) -> List[Document]:
#     """
#     여러 Wikidocs 형식의 텍스트 파일을 처리하여 Document 리스트로 반환
#     """
#     all_docs = []
#     for path in paths:
#         text = load_txt(path)
#         docs = split_by_chapter_with_code(text)
#         all_docs.extend(docs)
#     return all_docs

# def extract_and_replace_code_blocks(text: str):
#     """
#     <CODE_BLOCK>...</CODE_BLOCK> 구간을 [[CODE:0]], [[CODE:1]], ...로 치환
#     """
#     code_blocks = []
#     pattern = re.compile(r"<CODE_BLOCK>\s*```.*?\n.*?```\s*</CODE_BLOCK>", re.DOTALL)

#     def replacer(match):
#         code_blocks.append(match.group())
#         return f"[[CODE:{len(code_blocks) - 1}]]"

#     modified_text = pattern.sub(replacer, text)
#     return modified_text, code_blocks

# def split_protected_chunks(docs: List[Document], chunk_size=1000, chunk_overlap=200) -> List[Document]:
#     """
#     코드 블록이 잘리지 않도록 보호한 상태로 chunk 분할
#     """
#     splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap
#     )

#     chunked_docs = []
#     for doc in docs:
#         # 코드 블록을 임시 토큰으로 치환
#         mod_text, code_blocks = extract_and_replace_code_blocks(doc.page_content)
#         temp_doc = Document(page_content=mod_text, metadata=doc.metadata)

#         # 분할
#         split_docs = splitter.split_documents([temp_doc])

#         # 코드 블록 복원
#         for d in split_docs:
#             restored_text = d.page_content
#             for i, block in enumerate(code_blocks):
#                 restored_text = restored_text.replace(f"[[CODE:{i}]]", block)
#             d.page_content = restored_text
#             chunked_docs.append(d)

#     return chunked_docs

In [10]:
import re
from typing import List
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_txt(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def protect_code_blocks(text: str) -> str:
    code_pattern = re.compile(r"```.*?\n.*?```", re.DOTALL)
    protected = []
    last_end = 0
    for match in code_pattern.finditer(text):
        start, end = match.span()
        protected.append(text[last_end:start])
        code = match.group()
        protected.append(f"\n<CODE_BLOCK>\n{code}\n</CODE_BLOCK>\n")
        last_end = end
    protected.append(text[last_end:])
    return "".join(protected)

def split_by_structure_with_code(text: str) -> List[Document]:
    lines = text.splitlines()
    documents = []
    buffer = []

    current_chapter_num = None
    current_chapter_title = None
    current_section_num = None
    current_section_title = None

    chapter_pattern = re.compile(r"^---\s+CH(\d+)\s+(.*?)\s+---$")
    section_pattern = re.compile(r"^---\s+(\d{2})\.\s+(.*?)\s+---$")

    for line in lines:
        chapter_match = chapter_pattern.match(line)
        section_match = section_pattern.match(line)

        if chapter_match:
            # flush 이전 buffer
            if buffer:
                chunk = "\n".join(buffer).strip()
                protected = protect_code_blocks(chunk)
                documents.append(Document(
                    page_content=protected,
                    metadata={
                        "chapter_info": f"CH{current_chapter_num} {current_chapter_title}" if current_chapter_num else None,
                        "section_info": f"{current_section_num}. {current_section_title}" if current_section_num else None,
                    }
                ))
                buffer = []

            current_chapter_num = chapter_match.group(1)
            current_chapter_title = chapter_match.group(2).strip()
            current_section_num, current_section_title = None, None

        elif section_match:
            if buffer:
                chunk = "\n".join(buffer).strip()
                protected = protect_code_blocks(chunk)
                documents.append(Document(
                    page_content=protected,
                    metadata={
                        "chapter_info": f"CH{current_chapter_num} {current_chapter_title}" if current_chapter_num else None,
                        "section_info": f"{current_section_num}. {current_section_title}" if current_section_num else None,
                    }
                ))
                buffer = []

            current_section_num = section_match.group(1)
            current_section_title = section_match.group(2).strip()

        buffer.append(line)

    if buffer:
        chunk = "\n".join(buffer).strip()
        protected = protect_code_blocks(chunk)
        documents.append(Document(
            page_content=protected,
            metadata={
                "chapter_info": f"CH{current_chapter_num} {current_chapter_title}" if current_chapter_num else None,
                "section_info": f"{current_section_num}. {current_section_title}" if current_section_num else None,
            }
        ))

    return documents

def process_wikidocs_files(paths: List[str]) -> List[Document]:
    all_docs = []
    for path in paths:
        text = load_txt(path)
        docs = split_by_structure_with_code(text)
        all_docs.extend(docs)
    return all_docs

def extract_and_replace_code_blocks(text: str):
    code_blocks = []
    pattern = re.compile(r"<CODE_BLOCK>\s*```.*?\n.*?```\s*</CODE_BLOCK>", re.DOTALL)
    def replacer(match):
        code_blocks.append(match.group())
        return f"[[CODE:{len(code_blocks) - 1}]]"
    modified_text = pattern.sub(replacer, text)
    return modified_text, code_blocks

def split_protected_chunks(docs: List[Document], chunk_size=2000, chunk_overlap=50) -> List[Document]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )
    chunked_docs = []

    for doc in docs:
        mod_text, code_blocks = extract_and_replace_code_blocks(doc.page_content)
        temp_doc = Document(page_content=mod_text, metadata=doc.metadata)
        split_docs = splitter.split_documents([temp_doc])

        for d in split_docs:
            # 실제로 사용된 [[CODE:X]]만 찾아서 복원
            used_codes = re.findall(r"\[\[CODE:(\d+)\]\]", d.page_content)
            for code_index in set(used_codes):
                code_index = int(code_index)
                if code_index < len(code_blocks):
                    d.page_content = d.page_content.replace(f"[[CODE:{code_index}]]", code_blocks[code_index])
            chunked_docs.append(d)

    return chunked_docs

In [11]:
# 1. Wikidocs 텍스트 파일들을 챕터 단위로 로딩 (설명 + 코드 보존)
docs = process_wikidocs_files([
    "C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_01.txt",
    # "C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_02.txt",
    # "C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_03.txt"
])

# 2. 임베딩 전용 chunk 단위로 분할 (코드블록 보호됨)
chunked_docs = split_protected_chunks(docs)

# 3. 확인
print(f"총 원본 챕터 수: {len(docs)}")
print(f"총 분할된 문서 수 (임베딩용): {len(chunked_docs)}")
print("--- 샘플 ---")
print(chunked_docs[1].page_content[:500])


총 원본 챕터 수: 184
총 분할된 문서 수 (임베딩용): 1241
--- 샘플 ---
--- CH01 LangChain 시작하기 ---


In [12]:
# Document 객체에서 텍스트 내용만 추출
text_contents = [doc.page_content for doc in chunked_docs]

test_db = Chroma.from_texts(
    texts=text_contents,
    embedding=embeddings,
    persist_directory=r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\test_db"
)

In [13]:
test_db.as_retriever(search_type="similarity", search_kwargs={"k": 5}).invoke('랭체인')

[Document(metadata={}, page_content='--- 03. LangChain Hub ---'),
 Document(metadata={}, page_content='=== <랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 ===\n\n=================================================='),
 Document(metadata={}, page_content='--- CH01 LangChain 시작하기 ---'),
 Document(metadata={}, page_content='--- 05. 대화내용을 기억하는 RAG 체인 ---'),
 Document(metadata={}, page_content='--- 08. LCEL Chain 에 메모리 추가 ---')]

In [14]:
for doc in docs:
    print(f"문서 메타데이터:")
    for key, value in doc.metadata.items():
        if key == 'level':
            print(f"  {'  ' * (int(value) - 1)}└─ {key}: {value}")
        else:
            print(f"  {key}: {value}")
    print("-" * 50)

문서 메타데이터:
  chapter_info: None
  section_info: None
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: None
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: 01. 설치 영상보고 따라하기
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: 02. OpenAI API 키 발급 및 테스트
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: 03. LangSmith 추적 설정
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: 04. OpenAI API 사용(GPT-4o 멀티모달)
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: 05. LangChain Expression Language(LCEL)
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: 06. LCEL 인터페이스
---

In [15]:
# import re
# from langchain_core.documents import Document

# def extract_code_blocks_from_documents(docs: List[Document]) -> List[str]:
#     """
#     각 Document 객체 내의 <CODE_BLOCK>...</CODE_BLOCK> 구간만 추출하여 리스트로 반환
#     """
#     code_blocks = []
#     pattern = re.compile(r"<CODE_BLOCK>\s*```.*?\n.*?```\s*</CODE_BLOCK>", re.DOTALL)

#     for doc in docs:
#         matches = pattern.findall(doc.page_content)
#         code_blocks.extend(matches)

#     return code_blocks

# chunked_docs = split_protected_chunks(docs)

# code_blocks = extract_code_blocks_from_documents(chunked_docs)

# # 출력 확인
# for i, code in enumerate(code_blocks[100:150]):
#     print(f"🔹 Code Block {i + 1}:\n{code}\n{'-'*60}")


In [36]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# 1. 로컬 LLM (gemma:3.12b)
# llm = Ollama(model="gemma3:12b")
llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

# 4. 메타데이터 필드 정의 (사용 필드만)
metadata_field_info = [
    {"name": "chapter_info", "type": "string"},
    {"name": "section_info", "type": "string"},
]

# 5. ParentDocumentRetriever 구성
retriever = test_db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# 6. QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 7. 질문 실행
query = "랭체인이 뭔지 설명해"
result = qa_chain.invoke({"query": query})

# 8. 출력
print("🧠 답변:\n", result["result"])
print("\n📎 출처 문서:")
for doc in result["source_documents"]:
    meta = doc.metadata
    print(f"🔹 {meta.get('chapter_info', '정보 없음')} > {meta.get('section_info', '정보 없음')}")
    print(doc.page_content[:300], "\n---\n")

🧠 답변:
 랭체인(LangChain)은 언어 모델 기반 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크는 다양한 데이터 소스와 통합하여 복잡한 작업을 처리할 수 있도록 도와주며, 언어 모델의 기능을 맞춤 설정할 수 있는 도구와 컴포넌트를 제공합니다. 랭체인은 대화형 AI, 데이터 처리, 정보 검색 등 다양한 분야에서 활용될 수 있습니다.

📎 출처 문서:
🔹 정보 없음 > 정보 없음
=== <랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 ===

---

🔹 정보 없음 > 정보 없음
--- 05. 대화내용을 기억하는 RAG 체인 --- 
---

🔹 정보 없음 > 정보 없음
)
# 체인 생성
chain = prompt | llm | StrOutputParser()
# 질의 실행
response = chain.invoke({"question": "대한민국의 수도는 어디인가요?"})
대한민국(South Korea)의 수도는 서울입니다.
서울은 약 1000만 명의 인구를 가진 대도시로, 한반도 북부에 위치해 있습니다. 세계에서 가장 큰 도시 중 하나로 간주되며 문화, 경제 및 정치 중심지 역할을 하고 있습니다. 도시의 역사는 삼한 시대까지 거슬러 올라가며 이후 백제, 고려 그리고 조선 시대에 중요한 지역으로 
---

🔹 정보 없음 > 정보 없음
문서: data/SPRI_AI_Brief_2023년12월호_F.pdf (페이지 10)
LangSmith: https://smith.langchain.com/public/4449e744-f0a0-42d2-a3df-855bd7f41652/r
# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "삼성 가우스에 대해 설명해주세요"
response = rag_chain.invoke(question)
print(response)
삼성 가우스는 삼성전자가 개발한 생성 AI 모델로 
---

🔹 정보 없음 > 정보 없음
<CODE_B

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [5]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

In [3]:
loaders = [
    # 파일을 로드합니다.
    TextLoader(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\txt\wikidocs_01.txt"),
]

docs = []
for loader in loaders:
    # 로더를 사용하여 문서를 로드하고 docs 리스트에 추가합니다.
    docs.extend(loader.load())

In [8]:
# 자식 분할기를 생성합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)

# DB를 생성합니다.

vectorstore = Chroma(
    collection_name="test_01", embedding_function=HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
)

store = InMemoryStore()

# Retriever 를 생성합니다.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)


In [9]:
# 문서를 검색기에 추가합니다. docs는 문서 목록이고, ids는 문서의 고유 식별자 목록입니다.
retriever.add_documents(docs, ids=None, add_to_docstore=True)

# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Langchain")

In [10]:
sub_docs

[Document(id='5ca58a77-98d8-4874-a6b7-bf0e4c74bef0', metadata={'doc_id': '7738ffb4-e36b-43e2-8aa0-b00d465827ae', 'source': 'C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_01.txt'}, page_content='1. **Introduction to LangChain and LCEL**: LangChain offers a way to build complex chains from basic components, supporting functionalities like streaming, parallelism, and logging. LCEL (LangChain Expression Language) simplifies the process of chaining together prompts, models, and output parsers to perform tasks like generating jokes based on a given topic or conducting retrieval-augmented generation.'),
 Document(id='b20744d0-77c4-404e-a72b-d399c3c12604', metadata={'doc_id': '7738ffb4-e36b-43e2-8aa0-b00d465827ae', 'source': 'C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_01.txt'}, page_content='"langchain-kr/README.md at main · teddylee777/langchain-kr - GitHub", "url": "https://github.com/teddylee777/langchain-kr/blob/main/README.md"

In [12]:
print(sub_docs[0].page_content)

1. **Introduction to LangChain and LCEL**: LangChain offers a way to build complex chains from basic components, supporting functionalities like streaming, parallelism, and logging. LCEL (LangChain Expression Language) simplifies the process of chaining together prompts, models, and output parsers to perform tasks like generating jokes based on a given topic or conducting retrieval-augmented generation.


In [13]:
retrieved_docs = retriever.invoke("Langchain")

# 검색된 문서의 문서의 페이지 내용의 길이를 출력합니다.
print(
    f"문서의 길이: {len(retrieved_docs[0].page_content)}",
    end="\n\n=====================\n\n",
)

# 문서의 일부를 출력합니다.
print(retrieved_docs[0].page_content[2000:2500])

문서의 길이: 1800480


는 모듈식으로 설계되어, 사용하기 쉽습니다. 이는 개발자가 LangChain 프레임워크를 자유롭게 활용할 수 있게 해줍니다.
즉시 사용 가능한 체인 🚀
고수준 작업을 수행하기 위한 컴포넌트의 내장 조합을 제공합니다.
이러한 체인은 개발 과정을 간소화하고 속도를 높여줍니다.
주요 모듈 📌
모델 I/O 📃
프롬프트 관리, 최적화 및 LLM과의 일반적인 인터페이스와 작업을 위한 유틸리티를 포함합니다.
검색 📚
'데이터 강화 생성'에 초점을 맞춘 이 모듈은 생성 단계에서 필요한 데이터를 외부 데이터 소스에서 가져오는 작업을 담당합니다.
에이전트 🤖
언어 모델이 어떤 조치를 취할지 결정하고, 해당 조치를 실행하며, 관찰하고, 필요한 경우 반복하는 과정을 포함합니다.
LangChain을 활용하면, 언어 모델 기반 애플리케이션의 개발을 보다 쉽게 시작할 수 있으며, 필요에 맞게 기능을 맞춤 설정하고, 다양한 데이터 소스와 통합하여 복잡한 작업을 처리할 수 있게 됩니다.



In [20]:
# 부모 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 250, separators=['=================================================='])
# 자식 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
# 부모보다 작은 문서를 생성해야 합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 100)
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소입니다.
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
)
# 부모 문서의 저장 계층입니다.
store = InMemoryStore()

In [21]:
retriever = ParentDocumentRetriever(
    # 벡터 저장소를 지정합니다.
    vectorstore=vectorstore,
    # 문서 저장소를 지정합니다.
    docstore=store,
    # 하위 문서 분할기를 지정합니다.
    child_splitter=child_splitter,
    # 상위 문서 분할기를 지정합니다.
    parent_splitter=parent_splitter,
)

In [22]:
retriever.add_documents(docs)

# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Langchain")

# sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(sub_docs[0].page_content)

=== <랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 ===


In [23]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("Langchain")

# 검색된 문서의 첫 번째 문서의 페이지 내용의 길이를 반환합니다.
print(retrieved_docs[0].page_content)

=== <랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 ===


In [24]:
for doc in retrieved_docs[:5] :
    print(doc.page_content)

=== <랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 ===


--- 04. RAPTOR: 긴 문맥 요약(Long Context Summary) ---

```
Self-querying 방법은 자연어 쿼리를 받아 구조화된 쿼리를 작성하고, 이를 기반으로 VectorStore에 적용하여 문서의 의미적 유사성 비교 및 사용자 쿼리에서 추출한 필터를 메타데이터에 적용하여 실행하는 방식입니다. 예를 들어, Chroma vector store를 사용하여 영화 요약 문서가 포함된 작은 데모 세트를 생성하고, 이를 통해 자체 쿼리 검색기를 인스턴스화할 수 있습니다. 다음은 자체 쿼리 검색기를 사용하는 예시 코드입니다:
```python
%pip install --upgrade --quiet  lark chromadb
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
docs = [
Document(
page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
),
# Additional documents...
]
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI
meta

In [ ]:
loaders = [
    # 파일을 로드합니다.
    TextLoader(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\txt\pt_context.txt"),
]

docs = []
for loader in loaders:
    # 로더를 사용하여 문서를 로드하고 docs 리스트에 추가합니다.
    docs.extend(loader.load())

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 50,
    separators=['\n\n', '\n', '.', '']
)

split_docs = splitter.split_documents(docs)